In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import time
import pickle
import tensorflow as tf
import glob

In [2]:
features = ['TF(Term frequency) of body', 'TF of anchor', 'TF of title', 'TF of URL', 'TF of whole document', 'IDF(Inverse document frequency) of body', 
            'IDF of anchor', 'IDF of title', 'IDF of URL', 'IDF of whole document', 'TF*IDF of body', 'TF*IDF of anchor',
            'TF*IDF oftitle', 'TF*IDF of URL', 'TF*IDF of whole document', 'DL(Document length) of body' , 'DL of anchor', 'DL of title', 'DL of URL', 
            'DL of whole document', 'BM25 of body', 'BM25 of anchor', 'BM25 of title', 'BM25 of URL','BM25 of whole document', 
            'LMIR.ABS of body', 'LMIR.ABS of anchor', 'LMIR.ABS of title', 'LMIR.ABS of URL', 'LMIR.ABS of whole document', 'LMIR.DIR of body', 'LMIR.DIR of anchor',
            'LMIR.DIR of title', 'LMIR.DIR of URL', 'LMIR.DIR of whole document', 'LMIR.JM of body', 'LMIR.JM of anchor', 'LMIR.JM of title', 'LMIR.JM of URL', 
            'LMIR.JM of whole document', 'PageRank', 'Inlink number', 'Outlink number', 'Number of slash in URL', 'Length of URL', 'Number of child']

In [4]:
fold_num = 5

train = pd.read_csv('./data/MQ2008-list/I{}.txt'.format(fold_num),  sep=' ', header=None)

train = train.drop([48, 49, 51, 52,53, 54, 55, 56], axis=1)

columns = ['rank_num', 'query']

for i in range(len(features)):
    columns.append(features[i])

columns.extend(['doc_id'])
train.columns = columns

In [6]:
train.head()

,rank_num,query,TF(Term frequency) of body,TF of anchor,TF of title,TF of URL,TF of whole document,IDF(Inverse document frequency) of body,IDF of anchor,IDF of title,...,LMIR.JM of title,LMIR.JM of URL,LMIR.JM of whole document,PageRank,Inlink number,Outlink number,Number of slash in URL,Length of URL,Number of child,doc_id
0,1170,qid:18219,1:0.027615,2:0.500000,3:0.750000,4:0.333333,5:0.035088,6:0.000000,7:0.000000,8:0.000000,...,38:0.971101,39:0.863078,40:0.941432,41:0.222222,42:0.082873,43:0.000000,44:0.002255,45:0.010291,46:0.046400,GX004-93-7097963
1,1169,qid:18219,1:0.075314,2:0.875000,3:0.500000,4:0.166667,5:0.083542,6:0.000000,7:0.000000,8:0.000000,...,38:0.937646,39:0.963258,40:0.904074,41:0.111111,42:0.082873,43:0.000000,44:0.000377,45:0.004843,46:0.019200,GX009-58-16258917
2,1168,qid:18219,1:0.034310,2:0.375000,3:0.250000,4:0.333333,5:0.039265,6:0.000000,7:0.000000,8:0.000000,...,38:0.952023,39:0.863798,40:0.922098,41:0.333333,42:0.132597,43:0.000000,44:0.000257,45:0.010896,46:0.048000,GX016-32-14546147
3,1167,qid:18219,1:0.017573,2:0.375000,3:0.750000,4:0.166667,5:0.023392,6:0.000000,7:0.000000,8:0.000000,...,38:0.967764,39:0.815635,40:0.939661,41:0.111111,42:0.104972,43:0.000000,44:0.003897,45:0.007264,46:0.016000,GX011-24-8604590
4,1166,qid:18219,1:0.061925,2:0.125000,3:0.250000,4:0.166667,5:0.064327,6:0.000000,7:0.000000,8:0.000000,...,38:0.863175,39:0.905223,40:0.827869,41:0.444444,42:0.143646,43:0.000000,44:0.000184,45:0.000605,46:0.001600,GX065-10-5568515


In [14]:
train.shape

(180877, 49)

In [66]:
result = []
threshold = 204

start = time.time()

query_counter = 0

for query_key, group_df_query in train.groupby(['query']):
    q_id = query_key.split(':')[1]
    query_counter +=1
    tree = ET.parse('./data/MetaFeature/query-{}.xml'.format(q_id))
    q_meta = tree.getroot()
    q_value = []
    for i in range(1, int(q_meta[3][0].text)  + 1):
        q_value.append(str.encode(q_meta[3][i][1].text))
    result_dic = {}
    result_dic['query'] = q_value
    result_dic['doc_features'] = []
    result_dic['relevance'] = []
    result_dic['doc_keys'] = []
    counter = 0
    for doc_key, group_df_query_doc in group_df_query.groupby(['doc_id']):
        if (counter < threshold):
            #result_doc_dic = {}
            result_doc = []
            #result_doc_dic['doc_key'] = group_df_query_doc.values[0][-1]
            #result_doc_dic['rank_num'] =  group_df_query_doc.values[0][0]
            result_dic['relevance'].append(group_df_query_doc.values[0][0])
            #result_doc.append(group_df_query_doc.values[0][-1])
            result_dic['doc_keys'].append(str.encode(group_df_query_doc.values[0][-1]))
            
            for f_counter in range(2, 48):
                #result_doc_dic[columns[f_counter]] = group_df_query_doc.values[0][f_counter].split(':')[1]
                result_doc.append(float(group_df_query_doc.values[0][f_counter].split(':')[1]))
            
            #result_dic['docs'].append(result_doc_dic)
            result_dic['doc_features'].append(result_doc)
            counter +=1
    result.append(result_dic)

end = time.time()

In [17]:
(end - start) / 60

1.2381627082824707

In [68]:
pickle.dump(result, open( "processed_file.pickle", "wb" ))

In [8]:
doc_features = ['rank_num', 'doc_key', 'TF(Term frequency) of body', 'TF of anchor', 'TF of title', 'TF of URL', 'TF of whole document', 'IDF(Inverse document frequency) of body', 
            'IDF of anchor', 'IDF of title', 'IDF of URL', 'IDF of whole document', 'TF*IDF of body', 'TF*IDF of anchor',
            'TF*IDF oftitle', 'TF*IDF of URL', 'TF*IDF of whole document', 'DL(Document length) of body' , 'DL of anchor', 'DL of title', 'DL of URL', 
            'DL of whole document', 'BM25 of body', 'BM25 of anchor', 'BM25 of title', 'BM25 of URL','BM25 of whole document', 
            'LMIR.ABS of body', 'LMIR.ABS of anchor', 'LMIR.ABS of title', 'LMIR.ABS of URL', 'LMIR.ABS of whole document', 'LMIR.DIR of body', 'LMIR.DIR of anchor',
            'LMIR.DIR of title', 'LMIR.DIR of URL', 'LMIR.DIR of whole document', 'LMIR.JM of body', 'LMIR.JM of anchor', 'LMIR.JM of title', 'LMIR.JM of URL', 
            'LMIR.JM of whole document', 'PageRank', 'Inlink number', 'Outlink number', 'Number of slash in URL', 'Length of URL', 'Number of child']

In [69]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [70]:
def serialize_context(example):
    feature = {
            'query': _bytes_feature(example['query']),
    }

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    
    #return example_proto.SerializeToString()
    return example_proto

In [71]:
def serialize_example(example):
    feature = {
            'relevance': _int64_feature(example['relevance']),
            'doc_keys': _bytes_feature(example['doc_keys'])    
    }

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    
    #return example_proto.SerializeToString()
    return example_proto

In [72]:
data = {
    'context': serialize_context(result[0]),
    'examples': serialize_example(result[0])
}

In [76]:
a = ['a', 'b']

In [80]:
all_query_files = glob.glob("./data/MetaFeature/*.xml")

vocabulary = []

In [83]:
for query_file in all_query_files:
    tree = ET.parse(query_file)
    q_meta = tree.getroot()
    for i in range(1, int(q_meta[3][0].text)  + 1):
        word_val = q_meta[3][i][1].text
        if word_val not in vocabulary:
            vocabulary.append(word_val)

reading file ./data/MetaFeature/query-10154.xml
reading file ./data/MetaFeature/query-11272.xml
reading file ./data/MetaFeature/query-14785.xml
reading file ./data/MetaFeature/query-15803.xml
reading file ./data/MetaFeature/query-14964.xml
reading file ./data/MetaFeature/query-18143.xml
reading file ./data/MetaFeature/query-10652.xml
reading file ./data/MetaFeature/query-12987.xml
reading file ./data/MetaFeature/query-19493.xml
reading file ./data/MetaFeature/query-17323.xml
reading file ./data/MetaFeature/query-12861.xml
reading file ./data/MetaFeature/query-12220.xml
reading file ./data/MetaFeature/query-13893.xml
reading file ./data/MetaFeature/query-19108.xml
reading file ./data/MetaFeature/query-12829.xml
reading file ./data/MetaFeature/query-18021.xml
reading file ./data/MetaFeature/query-15851.xml
reading file ./data/MetaFeature/query-13899.xml
reading file ./data/MetaFeature/query-17548.xml
reading file ./data/MetaFeature/query-17581.xml
reading file ./data/MetaFeature/query-18

In [ ]:
pickle.dump(vocabulary, open( "vocabulary.pickle", "wb" ))